In [ ]:
# Setting
import os
import re
import numpy as np
import pandasas pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [ ]:
def load_data(file_path):
    # 데이터 로드 및 간단한 전처리
    path = "C:/Users/YY/Desktop/TB/Week05/NLP/"
    file = pd.read_csv(os.path.join(path, file_path), encoding='utf-8', index_col=0)
    # 필요 없는 칼럼 삭제 및 이름 변경
    file.drop(['from', 'Date'], axis=1, inplace=True)
    file.rename(columns={'x':'contents'}, inplace=True)
    print("loading done")
    return file

In [ ]:
def make_stopwords(file):
    # stopwords 준비
    lines = []
    f = open(os.path.join(path, file), 'r')
    while True:
        line = f.readline()
        if not line:
            break
        lines.append(line)
    f.close()
    stopwords = set(re.sub('\n', '', word) for word in lines)
    print(list(stopwords)[0:10])
    print("making stopwords done")
    return stopwords

In [ ]:
def remove_id():
    # 트위터 아이디를 제거해준다.
    pattern = re.compile('.@+[A-Za-z0-9\_]*:*')
    tweets = [re.sub(pattern, ' ', sentence) for sentence in list(file['contents'])]
    print("removing id done")
    return tweets
class TweetTokenizer:
    # 트윗을 토큰화함.
    def __init__(self):
        self.twitter = Twitter()
        self.stopwords = stopwords
    def nominalize(tweets, start, end):
        nouns = []
        for tweet in tweets[start:end]:
            nouns.append(' '.join([noun for noun in twitter.nouns(str(tweet)) if not noun in stopwords]))
            # print(len(nouns))
        # document = ' '.join(nouns)
        print("tokenizing done")
        return nouns
    

In [ ]:
def embedding_clustering():
    # Bag of Words를 만듦
    # 계산 비용 때문에 ngram을 (1, 1)로 max_features를 50으로 한정함
    vect = CountVectorizer(min_df=0.001, encoding='utf-8', max_features=50, ngram_range=(1, 1))
    bow = vect.fit_transform(nouns)
    print("사전 길이: ", len(vect.vocabulary_))
    # 학습을 위해 Dense Matrix로 바꿔줌
    X = bow.toarray()
    print("X shape: ", X.shape)
    vect.get_feature_names()
    dict = {'문재인':0, '남북정상회담':1, '지방선거':2, '자유한국당':3, '안철수':4, '더불어민주당':5,
            '미투':6, '바른미래당':7, '보수':8, '서울시장':9, '진보':10, '박원순':11, '김문수':12}
    # 실제 키워드: Y
    Y = np.array(file['Keyword'].map(dict)).astype(int).reshape(-1, 1)
    # 역시 계산 비용 때문에 kmeans외에 다른 클러스터링 방법을 적용하기 어려웠음
    kmeans = KMeans(n_clusters=13)
    kmeans.fit(X)
    # 예측 결과: pred
    pred = kmeans.predict(X).reshape(-1, 1)
    # 예측 결과 + 실제 결과: result
    result = np.concatenate([pred, Y], axis=1)
    print(pd.Series(pred.reshape(-1, )).value_counts())
    print(pd.Series(Y.reshape(-1, )).value_counts())
    return result

In [ ]:
def main():
    file = load_data('tweet.csv')
    stopwords = make_stopwords('korean_stopwords.txt')
    twitter = Twitter()
    tweets = remove_id()
    nouns = TweetTokenizer.nominalize(tweets, 0, 118570)
    result = embedding_clustering()
   

In [ ]:
 
# Run
if __name__ == '__main__':
    main()

In [ ]:
# 결과 분석 및 한계점 파악
# print를 통해 중간 결과 기록
print(list(stopwords)[0:10])
# ['이와같다면', '자', '그에 따르는', '영차', '얼마만큼', '양자', '막론하고', '아무도', '근거로', '이용하여']

In [ ]:
print("사전 길이: ", len(vect.vocabulary_))
# 사전 길이: 50
    
   

In [ ]:
 
print("X shape: ", X.shape)
# X shape: (118570, 50)
    
    
    
    
    

In [ ]:
# 비지도학습을 통한 군집 분류
print(pd.Series(pred.reshape(-1, )).value_counts())
# 7 25403
# 12 24670
# 1 11498
# 6 11219
# 10 10595
# 2 8117
# 0 8080
# 4 5044
# 11 3448
# 3 3084
# 8 3009
# 5 2612
# 9 1791
# dtype: int64
    
    

In [ ]:
    
    
# 실제 Keyword 분류
print(pd.Series(Y.reshape(-1, )).value_counts())
# 0 39300
# 1 17885
# 2 13530
# 3 10447
# 4 9834
# 5 7228
# 6 5391
# 7 4375
# 8 3602
# 9 2962
# 10 2381
# 11 1311
# 12 324
# dtype: int64